# Solve eigenvalue problem of the Laplacian by finite element method

On a square domain $\Omega$, consider the following eigenvalue problem

$$
-\Delta u = \lambda u \mbox{ in } \Omega, u=0 \mbox{ on } \partial \Omega \:.
$$

The variational formulation for the above eigenvalue problem is to find $u\in H_0^1(\Omega)$ and $\lambda \in R$ such that
$$
\int_{\Omega} \nabla u \cdot \nabla v dx = \lambda \int_{\Omega} uvdx \mbox{ for all }  v \in H_0^1(\Omega) \:.
$$

Below, we show how to solve the eigenvalue problem step by step.

Particular, in this speical case, the first 9 eigenvalue are given by 
$$
\left\{\frac{\lambda_i}{\pi^2} \right\}_{i=1}^{9}  = \{2,5,5,8,10,10,13,13,18\} \:.
$$

### Question:
    
Show the lower and upper bounds for the leading 9 exact eigenvalues.


<em>Last updated by Xuefeng LIU, Feb. 14, 2017</em>

## Step 1 : Mesh generation and FEM space definition

In [1]:
from dolfin import *

N=16; h=1.0/N
mesh = UnitSquareMesh(N, N)
V = FunctionSpace(mesh, "CG", 1)

# define Dirichlet boundary conditions
def bdry(x, on_boundary):  return on_boundary

bc = DirichletBC(V, Constant(0.0), bdry)

## Step 2: Variational formulation

In [2]:
# Define basis and bilinear form
u = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(u), grad(v))*dx
b = dot(u, v)*dx

L = v*dx # To feed an argument to assemble_system

# Assemble the stiffness matrix and the mass matrix.
A, _ = assemble_system(a, L, bc) #The assemble_system commands make sure the symmetry of A

B = assemble(b)

# set the diagonal elements of B corresponding to boundary nodes to zero to
# remove spurious eigenvalues.
#bc.zero(B)

## Step 3: Calculate matrix and solve the matrix eigenvalue problem

In [3]:
# downcast to PETSc matrices
MatA = as_backend_type(A)
MatB = as_backend_type(B)

# Create eigensolver
eigensolver = SLEPcEigenSolver(MatA, MatB)
# Specify the part of the spectrum desired
eigensolver.parameters["spectrum"] = "smallest magnitude"
# Specify the problem type (this can make a big difference)
eigensolver.parameters["problem_type"] = "gen_hermitian"
# Use the shift-and-invert spectral transformation
eigensolver.parameters["spectral_transform"] = "shift-and-invert"
# Specify the shift
eigensolver.parameters["spectral_shift"] = 1.0e-10

# Compute the eigenvalues. Here is where we specify the number of eigenvalues.
nreq = 9
eigensolver.solve(nreq)

## Step 4: Extract the eigenvalue and eigenfunction 

In [11]:
import numpy as np

exact_eigvalues = np.array([2,5,5,8,10,10,13,13,18])*pi**2;

# Extract the leading eigenpair from the smallest eigenvalue.
for k in range(0,9):
    r, c, rx, cx = eigensolver.get_eigenpair(k)
    exact_eig = exact_eigvalues[k]    
    print "The %dth approximate eigenvalue: %f=%8.3f*pi^2 (exact one:%4d*pi^2)"%(k+1,r, r/(pi**2), np.rint(exact_eig/(pi**2)))

The 1th approximate eigenvalue: 19.929513=   2.019*pi^2 (exact one:   2*pi^2)
The 2th approximate eigenvalue: 50.162062=   5.082*pi^2 (exact one:   5*pi^2)
The 3th approximate eigenvalue: 50.628396=   5.130*pi^2 (exact one:   5*pi^2)
The 4th approximate eigenvalue: 81.952730=   8.304*pi^2 (exact one:   8*pi^2)
The 5th approximate eigenvalue: 102.424349=  10.378*pi^2 (exact one:  10*pi^2)
The 6th approximate eigenvalue: 102.509086=  10.386*pi^2 (exact one:  10*pi^2)
The 7th approximate eigenvalue: 133.868267=  13.564*pi^2 (exact one:  13*pi^2)
The 8th approximate eigenvalue: 137.914570=  13.974*pi^2 (exact one:  13*pi^2)
The 9th approximate eigenvalue: 177.881329=  18.023*pi^2 (exact one:  18*pi^2)


## Solve the eignevalue by using sci sparse eigenvalue solver.

In [22]:
#Reference: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigsh.html

from PETSc2CSR import *
from scipy.sparse.linalg import eigsh

sA = PETSc2CSR(MatA)
sB = PETSc2CSR(MatB)
eigenvalues = eigsh(sA,k=6, M=sB,which="SM",return_eigenvectors=False,mode="normal")
eigenvalues.sort()
print(eigenvalues)

[  19.92951329   50.16206241   50.62839618   81.95272975  102.42434901
  102.509086  ]


## Step 5: Draw the eigenfunction

Please choose eig_index to be from 0 to 8 and check the shape of eigenfunction.

In [22]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

#Please choose the eige_index from 0 to 8.
eig_index=0
r, c, rx, cx = eigensolver.get_eigenpair(eig_index)

nodes = mesh.coordinates()
x = nodes[:,0]
y = nodes[:,1]

u = Function(V)
u.vector()[:] = rx 
z = u.compute_vertex_values(mesh)

fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot_trisurf(x, y, z, cmap=cm.jet, linewidth=0.2)
plt.show()

## Question:
    
Show the lower and upper bounds for the leading 9 exact eigenvalues.
